In [35]:
import csv
import math 
import random
import gzip
import torch
from sklearn import metrics
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os 
import warnings
warnings.filterwarnings("ignore")
import copy
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from einops import rearrange, reduce
from einops.layers.torch import Rearrange
from torch import einsum
import seaborn as sns
sns.set()

# DATA Preprocessing functions

In [36]:
def generate_chrom_train_test(peaks,Chr_dict):
    peak_chromosomes = {}
    n_peaks = 0
    chrom_train = []
    chrom_test = []
    for line in peaks:
        split_line = 'chromosome='+line.split('\t')[0]
        peak_chromosomes[split_line] = peak_chromosomes.get(split_line,0) + 1
        n_peaks+=1
    size_test = int(n_peaks/10)
    npeaks_test = 0
    while npeaks_test<size_test:
        chrom = random.choice(list(Chr_dict.keys()))
        npeaks_test += peak_chromosomes[chrom]
        chrom_test.append(chrom)
        if npeaks_test>int(n_peaks*1.3/10):
            npeaks_test = 0
            chrom_test = []
    for chrom in Chr_dict.keys():
        if chrom not in chrom_test:
            chrom_train.append(chrom)
    return chrom_train,chrom_test

In [37]:
def genome_data(data_file):
    data=open(data_file).read()
    chromosomes_data = data.split('>')[1:]
    return (chromosomes_data)

In [38]:
def Read_bed_file(chromosomes_data,peak_file,seq_length,Chr_dict,cross_chrom):
    peaks = open(peak_file).readlines()
    peak_sequences = []
    if cross_chrom:
        peak_sequences_train,peak_sequences_test = [],[]
        chrom_train , chrom_test = generate_chrom_train_test(peaks,Chr_dict)
    for peak in peaks:
        peak_split = peak.split('\t')
        Chr = 'chromosome='+str(peak_split[0])
        if Chr in Chr_dict:
            chrom_seq = Chr_dict[Chr]
            n = len(chrom_seq)
            start_idx = max(int(peak_split[1])-seq_length//2,0)
            end_idx = min(int(len(chrom_seq)), start_idx+seq_length)
            if end_idx == len(chrom_seq):
                start_idx = end_idx - seq_length
            header = Chr+':{0}-{1}'.format(start_idx,end_idx)
            if not cross_chrom:
                peak_sequences.append([header,chrom_seq[start_idx:end_idx]])
            else:
                if Chr in chrom_train:
                    peak_sequences_train.append([header,chrom_seq[start_idx:end_idx]])
                else:
                    peak_sequences_test.append([header,chrom_seq[start_idx:end_idx]])
    if not cross_chrom:
        size=int(len(peak_sequences)/10)
        peak_sequences_train = peak_sequences[:9*size]
        peak_sequences_test = peak_sequences[9*size:]
    return (peak_sequences_train,peak_sequences_test)

In [39]:
def dinucshuffle(sequence):
    b=[sequence[i:i+2] for i in range(0, len(sequence), 2)]
    random.shuffle(b)
    d=''.join([str(x) for x in b])
    return d

In [40]:
def seqtopad(sequence, motif_len):
    rows=len(sequence)+2*motif_len-2
    S=np.empty([rows,4])
    base=['A', 'C', 'G', 'T']
    for i in range(rows):
        for j in range(4):
            if (i-motif_len+1<len(sequence) and sequence[i-motif_len+1]=='N' 
                or i<motif_len-1 or i>len(sequence)+motif_len-2):
                S[i,j]=np.float32(0.25)
            elif sequence[i-motif_len+1]==base[j]:
                S[i,j]=np.float32(1)
            else:
                S[i,j]=np.float32(0)
    return np.transpose(S)

In [41]:
def generate_onehot_data(peak_sequences,motif_length,label,include_dinuc):
    alldata = []
    for header,seq in peak_sequences:
        alldata.append([header,seq,seqtopad(seq,motif_length),[int(label)]])#
        if include_dinuc:
            shuff_seq = dinucshuffle(seq)
            alldata.append([header,shuff_seq,seqtopad(shuff_seq,motif_length),[0]])#
    return (alldata)

In [42]:
def extract_data(data_file,peak_file,motif_length=24,seq_length=150,cross_chrom=False,include_dinuc=True,Chr_dict=None):
    chromosomes_data = genome_data(data_file)
    if Chr_dict==None:
        Chr_dict = {}
        for chrom_data in chromosomes_data:
            ref = chrom_data.split('\n')[0].split(' ')[-1][1:-1]
            Chr_dict[ref]=''.join(chrom_data.split('\n')[1:])
        Chr_dict['chromosome=Mito'] = Chr_dict.pop('top=circular')
    if type(peak_file) == str:
        peak_sequences_train,peak_sequences_test = Read_bed_file(chromosomes_data,peak_file,seq_length,Chr_dict,cross_chrom)
        train_data = generate_onehot_data(peak_sequences_train,motif_length,1,include_dinuc)
        test_data = generate_onehot_data(peak_sequences_test,motif_length,1,include_dinuc)
        random.shuffle(train_data)
        size=int(len(train_data)/10)
        calib_data=train_data[:9*size]
        valid_data=train_data[9*size:]
    elif type(peak_file) == list:
        include_dinuc = False
        peak_sequences_train,peak_sequences_test = [],[]
        train_data,test_data = [],[]
        for i in range (len(peak_file)):
            peak_sequences_train_temp,peak_sequences_test_temp = Read_bed_file(chromosomes_data,peak_file[i],seq_length,Chr_dict,cross_chrom)
            peak_sequences_train.extend(peak_sequences_train_temp)
            peak_sequences_test.extend(peak_sequences_test_temp)
            train_data.extend(generate_onehot_data(peak_sequences_train_temp,motif_length,i,include_dinuc))
            test_data.extend(generate_onehot_data(peak_sequences_test_temp,motif_length,i,include_dinuc))
        random.shuffle(train_data)
        random.shuffle(test_data)
        size=int(len(train_data)/10)
        calib_data=train_data[:9*size]
        valid_data=train_data[9*size:]
    return calib_data,valid_data,train_data,test_data,peak_sequences_train,peak_sequences_test

In [43]:
class dataset(Dataset):
    def __init__(self, xy=None):
        self.header=[el[0] for el in xy]
        self.seq =[el[1] for el in xy ]
        self.x_data=np.asarray([el[2] for el in xy],dtype=np.float32)
        self.y_data =np.asarray([el[3] for el in xy ],dtype=np.float32)
        self.x_data = torch.from_numpy(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.length=len(self.x_data)

    def __getitem__(self, index):
        return self.header[index],self.seq[index],self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.length

# Networks

In [44]:
class AttentionPool(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.to_attn_logits = nn.Parameter(torch.eye(dim)) 

    def forward(self, x):
        attn_logits = einsum('b n d, d e -> b n e', x, self.to_attn_logits) 
        attn = attn_logits.softmax(dim = -2) 
        return (x * attn).sum(dim = -2).squeeze()

In [45]:
class Network(nn.Module):
    def __init__(self, num_motif , motif_len , num_conv_layers , dropprob):
        super(Network, self).__init__()
        self.num_motif = num_motif
        self.conv = [nn.Conv1d(4, num_motif, kernel_size=motif_len),nn.ReLU(inplace=True)]
        in_channels = num_motif
        for i in range (num_conv_layers-1):
            motif_len = motif_len//2
            self.conv.append(nn.MaxPool1d(kernel_size=3))
            self.conv.append(nn.Conv1d(in_channels, int(1.5*in_channels), kernel_size=motif_len))
            self.conv.append(nn.ReLU(inplace=True))
            in_channels = int(1.5*in_channels)
        self.conv_layer = nn.Sequential(*self.conv)
        self.project = AttentionPool(in_channels)
        self.classifier = nn.Sequential(
            nn.Linear(in_channels , in_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropprob, inplace=False),
            nn.Linear(in_channels, 1),
            nn.Sigmoid())
    def forward(self, x):
        x = self.conv_layer(x)
        x= x.permute(0, 2, 1)
        x = self.project(x)
        predict = self.classifier(x)
        return predict

In [46]:
class conv_output(nn.Module):
    def __init__(self,filter_weights,filter_bias,device):
        super(conv_output, self).__init__()
        if type(filter_weights) is np.ndarray:
            self.filter_weights =  torch.from_numpy(filter_weights.astype(np.float32)).to(device)
        else :
            self.filter_weights = filter_weights.to(device)
        if type(filter_bias) is np.ndarray:
            self.filter_bias =  torch.from_numpy(filter_bias.astype(np.float32)).to(device)
        else :
            self.filter_bias = filter_bias.to(device)
        
    def forward(self,x):
        x=F.conv1d(x, self.filter_weights, bias=self.filter_bias, stride=1, padding=0)
        out=x.clamp(min=0)
        return (out)

In [47]:
### printing parameters ------------------
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table.get_string())
    print(f"\n Total Trainable Params: {total_params}")

# Calib - Train - Test functions 

In [48]:
def Train_model(model,train_loader,valid_loader, l_rate=0.01 , maxepochs=100,epochs_for_early_stop=0,save_model=False):
    best_model = None
    best_loss = np.inf
    counter = 0
    nepochs=0
    valid_losses =[]
    train_losses = []
    optimizer = torch.optim.AdamW(model.parameters(),lr=l_rate,weight_decay=1e-05)
    criterion = nn.BCELoss(reduction='mean')
    while nepochs<maxepochs:
        model.train()
        train_loss=0
        for i, (header, seq, data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)#
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
        if verbose:
            print('Model trained for {0} epochs out of {2}. Training loss is {1}'.format(nepochs+1,loss.item(),maxepochs))
        train_losses.append(train_loss/(i+1))
        with torch.no_grad():
            model.eval() 
            valid_loss=0
            for i, (header, seq, data, target) in enumerate(valid_loader):
                data = data.to(device)
                target = target.to(device)
                output = model(data)
                loss = F.binary_cross_entropy(output, target)#
                valid_loss+=loss.item()
            valid_losses.append(valid_loss/(i+1))
        counter+=1
        nepochs +=1
        if epochs_for_early_stop>0:
            if valid_losses[-1]<best_loss:
                if verbose:
                    print('Validation loss decreased from {0} to {1}'.format(best_loss,valid_losses[-1]))
                best_loss = valid_losses[-1]
                best_model = model
                counter = 0
            else:
                if verbose:
                    print('Counter for early stopping: {0} out of {1}'.format(counter,epochs_for_early_stop))
                if counter == epochs_for_early_stop:
                    print('early stopping at epoch ', nepochs-counter)
                    if save_model:
                        torch.save(best_model,model_dir+'/best_model.pkl')
                        count_parameters(best_model)
                    return (best_model,nepochs-counter,train_losses,valid_losses)
    print('no early stopping')
    if save_model:
        torch.save(best_model,model_dir+'/best_model.pkl')
        count_parameters(model)
    return (model,nepochs,train_losses,valid_losses)

In [49]:
def Test_model(model,test_loader):
    with torch.no_grad():
        model.eval()
        pred_list = []
        labels_list = []
        for i, (header, seq, data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            pred=output.cpu().detach().numpy().reshape(output.shape[0])
            labels=target.cpu().numpy().reshape(output.shape[0])
            pred_list.append(pred)
            labels_list.append(labels)
        labels = np.concatenate(labels_list)
        predictions = np.concatenate(pred_list)
    auc = metrics.roc_auc_score(labels, predictions)
    precision, recall, thresholds = metrics.precision_recall_curve(labels, predictions)
    prc = (metrics.auc(recall, precision))
    if verbose:
        print('AUROC on test data ', auc)
        print('AUPRC on test data ', prc)
    return (auc,prc)

In [50]:
def Calibrate_model(calib_loader,valid_loader, num_motif_list, num_conv_layers_list , dropprob_list,learning_rate_list, 
                    max_num_models=40, maxepochs=100,epochs_for_early_stop=0 , motif_len=24 ):
    results=pd.DataFrame(columns=['num_conv_layers','num_motif','Dropout','Learning Rate','epochs','AUROC','AUPRC'])
    best_AUC = 0
    if verbose:
        print('Training on ',device)
    for number in range(max_num_models):
        print('model {0} out of {1}'.format(number+1,max_num_models))
        # hyper-parameters
        num_motif = random.choice(num_motif_list)
        num_conv_layers = random.choice(num_conv_layers_list)
        dropprob = random.choice(dropprob_list)
        l_rate = random.choice(learning_rate_list)
        while ((results['num_conv_layers']==num_conv_layers) & (results['num_motif']==num_motif) 
               & (results['Dropout']==dropprob) & (results['Learning Rate']==l_rate)).any(): 
            #if hyperparameters exist in the results dataframe then randomly choose other parameters
            num_motif = random.choice(num_motif_list)
            num_conv_layers = random.choice(num_conv_layers_list)
            dropprob = random.choice(dropprob_list)
            l_rate = random.choice(learning_rate_list)
        model = Network(num_motif , motif_len , num_conv_layers , dropprob).to(device)#num_conv_layers,dropprob
        best_model,epochs,train_losses,valid_losses = Train_model(model,calib_loader,valid_loader,l_rate ,maxepochs,epochs_for_early_stop)
        auc,prc = Test_model(best_model,valid_loader)
        results=pd.concat([results,pd.DataFrame({'num_conv_layers':num_conv_layers,'num_motif':num_motif,'Dropout':dropprob,
                                                 'Learning Rate':l_rate,'epochs':epochs,'AUROC':[auc],'AUPRC':[prc]})])
        if auc > best_AUC :
            best_AUC = auc
            best_epochs = epochs
            best_num_motif = num_motif
            best_num_conv_layers = num_conv_layers
            best_dropprob = dropprob
            best_l_rate = l_rate
    best_hyperparameters = {'best_epochs': best_epochs,'best_num_motif':best_num_motif,
                            'best_num_conv_layers':best_num_conv_layers,'best_dropprob':best_dropprob,'best_l_rate':best_l_rate}
    torch.save(best_hyperparameters, model_dir+'/best_hyperpamarameters.pth')
    results.sort_values(by='AUROC',ascending=False,inplace=True)
    results.to_csv(model_dir+'/calibration_df.csv',index=False)
    return best_hyperparameters,results

In [51]:
def return_filter_outputs(model,test_loader):
    with torch.no_grad():
        best_model.eval()
        pred_list = []
        for i, (header, seq, data, target) in enumerate(test_loader):
            data = data.to(device)
            output = model(data)
            pred=output.cpu().detach().numpy()
            pred_list.append(pred)
        predictions = np.concatenate(pred_list)
    return predictions

# Motif extraction functions

In [52]:
def info_content(pwm, transpose=False, bg_gc=0.415):
    ''' Compute PWM information content.
    In the original analysis, I used a bg_gc=0.5. For any
    future analysis, I ought to switch to the true hg19
    value of 0.415.
    '''
    pseudoc = 1e-9
    if transpose:
        pwm = np.transpose(pwm)

    bg_pwm = [1-bg_gc, bg_gc, bg_gc, 1-bg_gc]

    ic = 0
    for i in range(pwm.shape[0]):
        for j in range(4):
            # ic += 0.5 + pwm[i][j]*np.log2(pseudoc+pwm[i][j])
            ic += -bg_pwm[j]*np.log2(bg_pwm[j]) + pwm[i][j]*np.log2(pseudoc+pwm[i][j])
    return ic

In [53]:
def meme_intro(meme_file, seqs):
    ''' Open MEME motif format file and print intro
    Attrs:
        meme_file (str) : filename
        seqs [str] : list of strings for obtaining background freqs
    Returns:
        mem_out : open MEME file
    '''
    nts = {'A':0, 'C':1, 'G':2, 'T':3}

    # count
    nt_counts = [1]*4
    for i in range(len(seqs)):
        for nt in seqs[i][1]:
            try:
                nt_counts[nts[nt]] += 1
            except KeyError:
                pass

    # normalize
    nt_sum = float(sum(nt_counts))
    nt_freqs = [nt_counts[i]/nt_sum for i in range(4)]

    # open file for writing
    meme_out = open(meme_file, 'w')

    # print intro material
    print( 'MEME version 4', file=meme_out)
    print( '', file=meme_out)
    #embd

    print( 'ALPHABET= ACGT', file=meme_out)        
    
    print( '', file=meme_out)
    print( 'Background letter frequencies:', file=meme_out)
    #embd
    print( 'A %.4f C %.4f G %.4f T %.4f' % tuple(nt_freqs), file=meme_out)
    print( '', file=meme_out)
    return meme_out


In [54]:
def make_filter_pwm(filter_fasta):
    ''' Make a PWM for this filter from its top hits '''
    nts = {'A':0, 'C':1, 'G':2, 'T':3}
    #embd
    pwm_counts = []
    nsites = 4 # pseudocounts
    for line in open(filter_fasta):
        if line[0] != '>':
            seq = line.rstrip()
            nsites += 1
            if len(pwm_counts) == 0:
                # initialize with the length
                for i in range(len(seq)):
                    pwm_counts.append(np.array([1.0]*4))

            # count
            for i in range(len(seq)):
                try:
                    pwm_counts[i][nts[seq[i]]] += 1
                except KeyError:
                    pwm_counts[i] += np.array([0.25]*4)

    # normalize
    pwm_freqs = []
    for i in range(len(pwm_counts)):
        pwm_freqs.append([pwm_counts[i][j]/float(nsites) for j in range(4)])

    return np.array(pwm_freqs), nsites-4

In [55]:
def plot_score_density(f_scores, out_pdf):
    sns.set(font_scale=1.3)
    plt.figure()
    sns.distplot(f_scores, kde=False)
    plt.xlabel('ReLU output')
    plt.savefig(out_pdf)
    plt.close()

    return f_scores.mean(), f_scores.std()

In [56]:
def filter_motif(param_matrix):
    nts = 'ACGT'
    motif_list = []
    for v in range(param_matrix.shape[1]):
        max_n = 0
        for n in range(1,4):
            if param_matrix[n,v] > param_matrix[max_n,v]:
                max_n = n

        if param_matrix[max_n,v] > 0:
            motif_list.append(nts[max_n])
        else:
            motif_list.append('N')

    return ''.join(motif_list)

In [57]:
def plot_filter_heat(param_matrix, out_pdf):
    param_range = abs(param_matrix).max()

    sns.set(font_scale=2)
    plt.figure(figsize=(param_matrix.shape[1], 4))
    sns.heatmap(param_matrix, cmap='PRGn', linewidths=0.2, vmin=-param_range, vmax=param_range)
    ax = plt.gca()
    ax.set_xticklabels(range(1,param_matrix.shape[1]+1))
    ax.set_yticklabels('ACGT', rotation='horizontal') # , size=10)
    plt.savefig(out_pdf)
    plt.close()

In [58]:
def plot_filter_logo(filter_outs, filter_size, seqs, out_prefix, raw_t=0, maxpct_t=None):
    if maxpct_t:
        all_outs = np.ravel(filter_outs)
        all_outs_mean = all_outs.mean()
        all_outs_norm = all_outs - all_outs_mean
        raw_t = 0.65 * all_outs_norm.max() + all_outs_mean
        #raw_t = 0.65 * all_outs_norm.max() + all_outs_mean
    # print fasta file of positive outputs
    filter_fasta_out = open('%s.fa' % out_prefix, 'w')
    filter_count = 0
    for i in range(filter_outs.shape[0]):
        for j in range(filter_outs.shape[1]):
            if filter_outs[i,j] > raw_t:
                #print(len(seqs[i]))
                fw.write(str(j))
                fw.write('\n')
                kmer = seqs[i][1][j:j+filter_size]
                #kmer = kmer.replace('T','U')
                incl_kmer = len(kmer) - kmer.count('N')
                if incl_kmer <filter_size:
                    continue
                print('>%d_%d' % (i,j), file=filter_fasta_out)
                print(kmer, file=filter_fasta_out)
                filter_count += 1
    filter_fasta_out.close()
    
    # make weblogo
    """if filter_count > 0:
        weblogo_cmd = 'weblogo %s < %s.fa > %s.eps' % (weblogo_opts, out_prefix, out_prefix)
        subprocess.call(weblogo_cmd, shell=True)"""

In [59]:
def meme_add(meme_out, f, filter_pwm, nsites, trim_filters=False):
    ''' Print a filter to the growing MEME file
    Attrs:
        meme_out : open file
        f (int) : filter index #
        filter_pwm (array) : filter PWM array
        nsites (int) : number of filter sites
    '''
    if not trim_filters:
        ic_start = 0
        ic_end = filter_pwm.shape[0]-1
    else:
        ic_t = 0.2

        # trim PWM of uninformative prefix
        ic_start = 0
        while ic_start < filter_pwm.shape[0] and info_content(filter_pwm[ic_start:ic_start+1]) < ic_t:
            ic_start += 1

        # trim PWM of uninformative suffix
        ic_end = filter_pwm.shape[0]-1
        while ic_end >= 0 and info_content(filter_pwm[ic_end:ic_end+1]) < ic_t:
            ic_end -= 1

    if ic_start < ic_end:
        print('MOTIF filter%d' % f, file=meme_out)
        print('letter-probability matrix: alength= 4 w= %d nsites= %d' % (ic_end-ic_start+1, nsites), file=meme_out)

        for i in range(ic_start, ic_end+1):
            print( '%.4f %.4f %.4f %.4f' % tuple(filter_pwm[i]), file=meme_out)


        print( '', file=meme_out)

In [60]:
def plot_filter_heat(param_matrix, out_pdf):
    param_range = abs(param_matrix).max()

    sns.set(font_scale=2)
    plt.figure(figsize=(param_matrix.shape[1], 4))
    sns.heatmap(param_matrix, cmap='PRGn', linewidths=0.2, vmin=-param_range, vmax=param_range)
    ax = plt.gca()
    ax.set_xticklabels(range(1,param_matrix.shape[1]+1))
    ax.set_yticklabels('ACGT', rotation='horizontal') # , size=10)
    plt.savefig(out_pdf)
    plt.close()

In [61]:
def get_motif(filter_weights_old, filter_outs, seqs, out_dir):
    global fw
    filter_weights = []
    for x in filter_weights_old:
        x = x - np.mean(x,axis = 0)
        filter_weights.append(x)
        
    filter_weights = np.array(filter_weights)
    num_filters = filter_weights.shape[0]
    filter_size = filter_weights.shape[2]
    filters_ic = []
    meme_out = meme_intro('%s/filters_meme.txt'%(out_dir), seqs)
    fw = open('indices.txt', 'w')
    for f in range(num_filters):
        # plot filter parameters as a heatmap
        plot_filter_heat(filter_weights[f,:,:filter_size], '%s/filter%d_heat.pdf' % (out_dir,f))

        # plot weblogo of high scoring outputs
        plot_filter_logo(filter_outs[:,f,:], filter_size, seqs, '%s/filter%d_logo'%(out_dir,f), maxpct_t=0.5)

        # make a PWM for the filter
        filter_pwm, nsites = make_filter_pwm('%s/filter%d_logo.fa'%(out_dir,f))

        if nsites < 10:
            # no information
            filters_ic.append(0)
        else:
            # compute and save information content
            filters_ic.append(info_content(filter_pwm))

            # add to the meme motif file
            meme_add(meme_out, f, filter_pwm, nsites, False)

    meme_out.close()
    fw.close()
    table_out = open('%s/table.txt'%out_dir, 'w')

    # print header for later panda reading
    table = PrettyTable(["Filter", "consensus","annotation","ic","mean","std"])
    header_cols = ('', 'consensus', 'annotation', 'ic', 'mean', 'std')
    print('%3s  %19s  %10s  %5s  %6s  %6s' % header_cols, file=table_out)
    
    for f in range(num_filters):
        # collapse to a consensus motif
        consensus = filter_motif(filter_weights[f,:,:])

        # grab annotation
        annotation = '.'

        # plot density of filter output scores
        fmean, fstd = plot_score_density(np.ravel(filter_outs[: , f , :]), '%s/filter%d_dens.pdf' % (out_dir,f))

        row_cols = (f, consensus, annotation, filters_ic[f], fmean, fstd)
        table.add_row(list(row_cols))
        print( '%-3d  %19s  %10s  %5.2f  %6.4f  %6.4f' % row_cols, file=table_out)
        
    table_out.close()
    print(table.get_string())

# Main

In [62]:
global device
global model_dir
global results_dir
global data_dir
global verbose
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_dir = 'model'
results_dir = 'results'
data_dir = 'Data'
verbose = False
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

In [63]:
data_file = data_dir+'/S288C_reference_sequence_R64-3-1_20210421.fsa'
peak_file = data_dir+'/Condensin_peaks_Log.bed'#data_dir+'/Condensin_peaks_quiescence.bed'
seq_length = 500
motif_length = 24
cross_chrom = True
include_dinuc = True
calib_data,valid_data,train_data,test_data,peak_sequences_train,peak_sequences_test = extract_data(data_file,peak_file,motif_length,
                                                                                                   seq_length,cross_chrom,include_dinuc)
len(calib_data),len(valid_data),len(train_data),len(test_data)

(1575, 183, 1758, 226)

In [64]:
calib_dataset=dataset(calib_data)
valid_dataset=dataset(valid_data)
train_dataset=dataset(train_data)
test_dataset=dataset(test_data)
batch_size = 64
calib_loader = DataLoader(dataset=calib_dataset,
                          batch_size=batch_size,shuffle=False)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch_size,shuffle=False)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,shuffle=False)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,shuffle=False)

In [65]:
num_motif_list = [30,40,60]
num_conv_layers_list = [1,2]
dropprob_list = [0, 0.15, 0.3]
learning_rate_list = [10**-5,10**-4,10**-3,10**-2]
max_num_models = 30
maxepochs = 500
epochs_for_early_stop = 50
best_hyperparameters,results = Calibrate_model(calib_loader,valid_loader, num_motif_list, num_conv_layers_list , dropprob_list,
                                               learning_rate_list, max_num_models=max_num_models, maxepochs=maxepochs,
                                               epochs_for_early_stop=epochs_for_early_stop,motif_len=motif_length)

model 1 out of 30
early stopping at epoch  1
model 2 out of 30
early stopping at epoch  12
model 3 out of 30
early stopping at epoch  6
model 4 out of 30
early stopping at epoch  244
model 5 out of 30
early stopping at epoch  33
model 6 out of 30
early stopping at epoch  27
model 7 out of 30
early stopping at epoch  346
model 8 out of 30
early stopping at epoch  1
model 9 out of 30
early stopping at epoch  282
model 10 out of 30
no early stopping
model 11 out of 30
early stopping at epoch  12
model 12 out of 30
early stopping at epoch  1
model 13 out of 30
early stopping at epoch  290
model 14 out of 30
no early stopping
model 15 out of 30
no early stopping
model 16 out of 30
early stopping at epoch  218
model 17 out of 30
early stopping at epoch  6
model 18 out of 30
early stopping at epoch  43
model 19 out of 30
no early stopping
model 20 out of 30
early stopping at epoch  43
model 21 out of 30
early stopping at epoch  48
model 22 out of 30
early stopping at epoch  427
model 23 out o

In [79]:
results.head()

,num_conv_layers,num_motif,Dropout,Learning Rate,epochs,AUROC,AUPRC
0,2,60,0.15,0.0001,282,0.976293,0.977569
0,2,40,0.15,0.0001,218,0.963482,0.969429
0,2,60,0.15,0.0010,33,0.960010,0.968901
0,1,60,0.00,0.0001,290,0.959531,0.965703
0,2,30,0.15,0.0001,244,0.954502,0.961075


In [67]:
best_hyperparameters

{'best_epochs': 282,
 'best_num_motif': 60,
 'best_num_conv_layers': 2,
 'best_dropprob': 0.15,
 'best_l_rate': 0.0001}

In [68]:
maxepochs = best_hyperparameters['best_epochs']
num_motif = best_hyperparameters['best_num_motif']
num_conv_layers = best_hyperparameters['best_num_conv_layers']
dropprob = best_hyperparameters['best_dropprob']
l_rate = best_hyperparameters['best_l_rate']
epochs_for_early_stop = 0
model = Network(num_motif , motif_length , num_conv_layers , dropprob).to(device)
best_model,epochs,train_losses,valid_losses = Train_model(model,train_loader,valid_loader,l_rate ,maxepochs,
                                                          epochs_for_early_stop,save_model=True)

no early stopping
+------------------------+------------+
|        Modules         | Parameters |
+------------------------+------------+
|  conv_layer.0.weight   |    5760    |
|   conv_layer.0.bias    |     60     |
|  conv_layer.3.weight   |   64800    |
|   conv_layer.3.bias    |     90     |
| project.to_attn_logits |    8100    |
|  classifier.0.weight   |    8100    |
|   classifier.0.bias    |     90     |
|  classifier.3.weight   |     90     |
|   classifier.3.bias    |     1      |
+------------------------+------------+

 Total Trainable Params: 87091


In [69]:
auc,prc = Test_model(best_model,test_loader)
auc,prc

(0.9476074868822931, 0.9574728745148315)

In [70]:
weights = best_model.conv[0].weight.detach().cpu().numpy()
bias = best_model.conv[0].bias.detach().cpu().numpy()
motif_sequences=generate_onehot_data(peak_sequences_test,motif_length,1,include_dinuc=False)
motif_dataset=dataset(motif_sequences)
motif_loader = DataLoader(dataset=motif_dataset,
                          batch_size=batch_size,shuffle=False)
out_model = conv_output(weights,bias,device)
filter_output = return_filter_outputs(out_model,motif_loader)

In [71]:
weights.shape

(60, 4, 24)

In [72]:
filter_output.shape

(113, 60, 523)

# Motif extraction

In [73]:
get_motif(weights,filter_output,peak_sequences_test,results_dir)

+--------+--------------------------+------------+-----------------------+-------------+-------------+
| Filter |        consensus         | annotation |           ic          |     mean    |     std     |
+--------+--------------------------+------------+-----------------------+-------------+-------------+
|   0    | AAAACAATTAGACTGAAAAAGCTG |     .      |  0.17864202909730897  |  0.20100093 |  0.2292801  |
|   1    | AGCTGCTGCCTAAATGTATCATGC |     .      |   1.0209365066142762  |  0.24851735 |  0.23527859 |
|   2    | CGCCGCGAGAAATCATTAATCGCT |     .      |   1.0348282778471212  |  0.1477161  |  0.1841126  |
|   3    | GCGACGGGGTGCGACGAGCCCCCC |     .      |  0.03037135849468614  |  0.14012165 |  0.16896287 |
|   4    | AAGTACGGCCGACGCACTGAATCG |     .      |   0.5945036300333117  | 0.100685075 |  0.14922039 |
|   5    | TAATGGACAATAGGGTACGGCTAG |     .      |  0.23678172330564856  |   0.118984  |  0.18848166 |
|   6    | TAAAGCCTTACGGCTTATTGAGAC |     .      |   0.506387683376536   